# SETE
Sequence-based Ensemble learning approach for TCR Epitope binding prediction

In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from scipy import interp
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import SGDClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Concatenate, Input
from tensorflow.keras.optimizers import Adam

import warnings

warnings.filterwarnings('ignore')

2024-12-13 15:29:01.407368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734128941.421263 1821575 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734128941.425383 1821575 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 15:29:01.441246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_file_path = '../data/BAP/tcr_split/train.csv'

train_df = pd.read_csv(train_file_path, header=None)
train_df.columns = ['epitope', 'tcr', 'label']

train_df['epitope'] = train_df['epitope'].str.upper()
train_df['tcr'] = train_df['tcr'].str.upper()

print(train_df.shape)
train_df.head()

(191897, 3)


,epitope,tcr,label
0,EAAGIGILTV,CASSQEEGGGSWGNTIYF,1
1,EAAGIGILTV,CASSQGQLTDTQYF,1
2,EAAGIGILTV,CASSRTVGGPNEQF,1
3,EAAGIGILTV,CASSTGQGWGSF,1
4,SAYGEPRKL,CASLAGQGYNEQF,1


In [3]:
test_file_path = '../data/BAP/tcr_split/test.csv'

test_df = pd.read_csv(test_file_path, header=None)
test_df.columns = ['epitope', 'tcr', 'label']

test_df['epitope'] = test_df['epitope'].str.upper()
test_df['tcr'] = test_df['tcr'].str.upper()

print(test_df.shape)
test_df.head()

(48042, 3)


,epitope,tcr,label
0,EAAGIGILTV,CASSQEGLAGASQYF,1
1,EAAGIGILTV,CASSQETDIVFNOPQHF,1
2,EAAGIGILTV,CSTDGQTGTGELF,1
3,AVFDRKSDAK,CATGPRANTGELF,1
4,GLCTLVAML,CASSSGELSAGELF,1


In [4]:
def data_preprocess(df, k=2, remove_duplicate=False, min_tcrs_amount=1):
        
    if remove_duplicate:
        df.drop_duplicates(subset=['epitope', 'tcr'], inplace=True)

    # Group by epitope and filter based on min_tcrs_amount
    epitope_counts = df.groupby('epitope').size()
    valid_epitopes = epitope_counts[epitope_counts >= min_tcrs_amount].index
    df = df[df['epitope'].isin(valid_epitopes)]

    alphabet = ['A', 'B', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 
                'X', 'B', 'J', 'Z', 'U', 'O']

    # Generate all k-letter combinations (with repetition allowed)
    k_letter_combinations = list(itertools.product(alphabet, repeat=k))

    # Convert tuples to strings
    k_letter_sequences = [''.join(comb) for comb in k_letter_combinations]
    
    print(f"Distinct kmers: {len(k_letter_sequences)}")
    
    # Convert list to dictionary
    sequence_to_index_map = {sequence: index+1 for index, sequence in enumerate(k_letter_sequences)}
    # print(sequence_to_index_map)

    X = np.zeros((len(df), len(k_letter_sequences)*2 + 2 + 200))
    y = df['label'].values

    def get_kmer_indices(sequence, k, sequence_to_index_map):
        indices = []
        for i in range(len(sequence) - k + 1):
            kmer = sequence[i:i+k]
            if kmer in sequence_to_index_map:
                indices.append(sequence_to_index_map[kmer])
            else:
                indices.append(-1)  # If the kmer is not found, use -1 (or any other placeholder)
        return indices

    # Process each row
    for i, (_, row) in enumerate(df.iterrows()):
        epitope = row['epitope']
        tcr = row['tcr']
        
        # Get k-mer indices for epitope and tcr
        epitope_indices = get_kmer_indices(epitope, k, sequence_to_index_map)
        tcr_indices = get_kmer_indices(tcr, k, sequence_to_index_map) 
        
        # Pad sequences to length 100
        epitope_indices_padded = epitope_indices[:100] + [0] * (100 - len(epitope_indices))
        tcr_indices_padded = tcr_indices[:100] + [0] * (100 - len(tcr_indices))

        # Initialize the count array
        epitope_counts = np.zeros(len(k_letter_sequences), dtype=int)

        # Lambda function to process the epitope string in chunks of size k and update counts
        process_kmers_epitope = lambda epitope: [
            epitope_counts.__setitem__(sequence_to_index_map[epitope[j:j + k]], epitope_counts[sequence_to_index_map[epitope[j:j + k]]] + 1)
            for j in range(len(epitope) - k + 1) 
        ]
        
        process_kmers_epitope(epitope)
        epitope_counts = np.append(epitope_counts, len(epitope))

        # Initialize the count array
        tcr_counts = np.zeros(len(k_letter_sequences), dtype=int)

        # Lambda function to process the epitope string in chunks of size k and update counts
        process_kmers_tcr = lambda tcr: [
            tcr_counts.__setitem__(sequence_to_index_map[tcr[j:j + k]], tcr_counts[sequence_to_index_map[tcr[j:j + k]]] + 1)
            for j in range(len(tcr) - k + 1) 
        ]
        
        process_kmers_tcr(tcr)    
        tcr_counts = np.append(tcr_counts, len(tcr))
        
        X[i] = np.concatenate([np.array(epitope_indices_padded), np.array(tcr_indices_padded), epitope_counts, tcr_counts])
    
    return X, y

In [5]:
X, y = data_preprocess(train_df, k=2, min_tcrs_amount=1)
print(X.shape)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Distinct kmers: 729
(191897, 1660)


In [6]:
X_test, y_test = data_preprocess(test_df, k=2, min_tcrs_amount=1)
print(X_test.shape)

Distinct kmers: 729
(48042, 1660)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [8]:
# pca = PCA(n_components=200)
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_val_pca = pca.transform(X_val_scaled)
# X_test_pca = pca.transform(X_test_scaled)

In [9]:
# Not going with PCA as the results on test data is poor

X_train_pca = X_train_scaled
X_val_pca = X_val_scaled
X_test_pca = X_test_scaled

In [10]:
# X = pd.read_csv('./embeddings/X_features_epi_split_train.csv').values
# y = pd.read_csv('./embeddings/y_labels_epi_split_train.csv')['label'].values
# X.shape

In [11]:
# pd.DataFrame(X).to_csv('./embeddings/epi_split_train_X_k2.csv', index=False)
# pd.DataFrame(y, columns=['label']).to_csv('./embeddings/epi_split_train_y.csv', index=False)

In [12]:
train_dmatrix = xgb.DMatrix(X_train_pca, label=y_train)
val_dmatrix = xgb.DMatrix(X_val_pca)

# Define parameters
params = {
    'objective': 'binary:logistic',  
    'eval_metric': 'logloss',       
    'max_depth': 7,                
    'eta': 0.2,                    
    'seed': 42
}

# Train model
xgb_model = xgb.train(params, train_dmatrix, num_boost_round=500)

y_pred_proba_train = xgb_model.predict(train_dmatrix)
y_pred_train = [1 if prob > 0.5 else 0 for prob in y_pred_proba_train]

y_pred_proba_val = xgb_model.predict(val_dmatrix)
y_pred_val = [1 if prob > 0.5 else 0 for prob in y_pred_proba_val]

print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))

Training Accuracy: 0.86678348326244
Validation Accuracy: 0.7272016675351746


In [13]:
gb_model = GradientBoostingClassifier(learning_rate=0.1,
                                        min_samples_leaf=20, max_features='sqrt', subsample=0.8,
                                        random_state=42, n_estimators=70, max_depth=11,
                                        min_samples_split=60, loss="log_loss", verbose=1
                                        )

gb_model.fit(X_train_pca, y_train)

y_pred_train = gb_model.predict(X_train_pca)
y_pred_val = gb_model.predict(X_val_pca)

print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3769           0.0087           31.37s
         2           1.3601           0.0163           30.97s
         3           1.3457           0.0158           30.30s
         4           1.3349           0.0095           29.84s
         5           1.3241           0.0115           29.38s
         6           1.3145           0.0082           28.84s
         7           1.3046           0.0100           28.24s
         8           1.2920           0.0133           27.83s
         9           1.2809           0.0085           27.42s
        10           1.2730           0.0095           26.94s
        20           1.2151           0.0005           22.39s
        30           1.1790           0.0108           17.98s
        40           1.1533           0.0069           13.44s
        50           1.1360           0.0014            8.97s
        60           1.1181          -0.0060            4.50s
       

In [14]:
# Split the features into temporal and non-temporal
temporal_features = 200  # First 200 features are temporal
X_train_temporal = X_train_pca[:, :temporal_features]  # Temporal features
X_train_non_temporal = X_train_pca[:, temporal_features:]  # Non-temporal features

X_val_temporal = X_val_pca[:, :temporal_features]  # Temporal features
X_val_non_temporal = X_val_pca[:, temporal_features:]  # Non-temporal features

# Reshape temporal features to be 3D [samples, time steps, features]
X_train_temporal_reshaped = X_train_temporal.reshape((X_train_temporal.shape[0], 100, 2))
X_val_temporal_reshaped = X_val_temporal.reshape((X_val_temporal.shape[0], 100, 2))  

# Build the model
temporal_input = Input(shape=(100, 2))  # Input for temporal features
temporal_branch = LSTM(256, return_sequences=True)(temporal_input)  # LSTM for temporal features
temporal_branch = Dropout(0.3)(temporal_branch)
temporal_branch = LSTM(128, return_sequences=True)(temporal_branch)
temporal_branch = Dropout(0.3)(temporal_branch)
temporal_branch = LSTM(64)(temporal_branch)  # Final LSTM layer for temporal data

# Non-temporal input
non_temporal_input = Input(shape=(X_train_non_temporal.shape[1],))  # Input for non-temporal features
non_temporal_branch = Dense(128, activation='relu')(non_temporal_input)
non_temporal_branch = Dropout(0.3)(non_temporal_branch)
non_temporal_branch = Dense(64, activation='relu')(non_temporal_branch)

# Concatenate temporal and non-temporal branches
merged = Concatenate()([temporal_branch, non_temporal_branch])

# Final Dense layers
dense_layer = Dense(128, activation='relu')(merged)
dense_layer = Dropout(0.3)(dense_layer)
dense_layer = Dense(64, activation='relu')(dense_layer)
output = Dense(1, activation='sigmoid')(dense_layer)

# Compile the model
nn_model = Model(inputs=[temporal_input, non_temporal_input], outputs=output)
nn_model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

# Train the model
history = nn_model.fit([X_train_temporal_reshaped, X_train_non_temporal], y_train,
                       epochs=15,
                       batch_size=64,
                       validation_split=0.15,
                       verbose=1)

# Predictions
y_pred_proba_train = nn_model.predict([X_train_temporal_reshaped, X_train_non_temporal])
y_pred_train = [1 if prob > 0.5 else 0 for prob in y_pred_proba_train]

y_pred_proba_val = nn_model.predict([X_val_temporal_reshaped, X_val_non_temporal])
y_pred_val = [1 if prob > 0.5 else 0 for prob in y_pred_proba_val]

# Evaluate accuracy
print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))


I0000 00:00:1734129138.583682 1821575 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22398 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:e2:00.0, compute capability: 8.0


Epoch 1/15


I0000 00:00:1734129143.857438 1821847 cuda_dnn.cc:529] Loaded cuDNN version 90300


2039/2039 ━━━━━━━━━━━━━━━━━━━━ 37s 16ms/step - accuracy: 0.6114 - loss: 0.6473 - val_accuracy: 0.6793 - val_loss: 0.5832
Epoch 2/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.6829 - loss: 0.5801 - val_accuracy: 0.6922 - val_loss: 0.5683
Epoch 3/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.6950 - loss: 0.5618 - val_accuracy: 0.6978 - val_loss: 0.5609
Epoch 4/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.7117 - loss: 0.5446 - val_accuracy: 0.7018 - val_loss: 0.5507
Epoch 5/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.7149 - loss: 0.5349 - val_accuracy: 0.7041 - val_loss: 0.5521
Epoch 6/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.7226 - loss: 0.5258 - val_accuracy: 0.7064 - val_loss: 0.5473
Epoch 7/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.7262 - loss: 0.5199 - val_accuracy: 0.7095 - val_loss: 0.5450
Epoch 8/15
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.7329 - loss: 0.51

In [15]:
#Performance on Test Data

test_dmatrix = xgb.DMatrix(X_test_pca)
y_pred_proba_test = xgb_model.predict(test_dmatrix)
y_pred_test_xgb = [1 if prob > 0.5 else 0 for prob in y_pred_proba_test]

y_pred_test_gb = gb_model.predict(X_test_pca)

X_test_temporal = X_test_pca[:, :temporal_features]  # Temporal features
X_test_non_temporal = X_test_pca[:, temporal_features:]  # Non-temporal features
X_test_temporal_reshaped = X_test_temporal.reshape((X_test_temporal.shape[0], 100, 2)) 
y_pred_proba_test = nn_model.predict([X_test_temporal_reshaped, X_test_non_temporal])
y_pred_test_nn = [1 if prob > 0.5 else 0 for prob in y_pred_proba_test]

print("XG Boost")
print("Test Accuracy:", accuracy_score(y_test, y_pred_test_xgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_test_xgb))
auc = roc_auc_score(y_test, y_pred_test_xgb)
print("AUC: ", auc)

print("Gradient Boosting Classifier")
print("Test Accuracy:", accuracy_score(y_test, y_pred_test_gb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_test_gb))
auc = roc_auc_score(y_test, y_pred_test_gb)
print("AUC: ", auc)

print("Neural Network")
print("Test Accuracy:", accuracy_score(y_test, y_pred_test_nn))
print("\nClassification Report:\n", classification_report(y_test, y_pred_test_nn))
auc = roc_auc_score(y_test, y_pred_test_nn)
print("AUC: ", auc)

1502/1502 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step
XG Boost
Test Accuracy: 0.7282794221722659

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.77      0.74     24076
           1       0.75      0.69      0.72     23966

    accuracy                           0.73     48042
   macro avg       0.73      0.73      0.73     48042
weighted avg       0.73      0.73      0.73     48042

AUC:  0.7281888737072097
Gradient Boosting Classifier
Test Accuracy: 0.6973065234586403

Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.75      0.71     24076
           1       0.72      0.65      0.68     23966

    accuracy                           0.70     48042
   macro avg       0.70      0.70      0.70     48042
weighted avg       0.70      0.70      0.70     48042

AUC:  0.6971944609961859
Neural Network
Test Accuracy: 0.7091295116772823

Classification Report:
               precision

In [17]:
# Predict on hold-out set

holdout_file_path = '../data/true_hold_out/tcr_split_test.csv'

holdout_df = pd.read_csv(holdout_file_path, header=None)
holdout_df.columns = ['epitope', 'tcr']

holdout_df['epitope'] = holdout_df['epitope'].str.upper()
holdout_df['tcr'] = holdout_df['tcr'].str.upper()

# default value to satisfy preprocess function definition
holdout_df['label'] = 0

print(holdout_df.shape)
print(holdout_df.head())

X_holdout, y_holdout = data_preprocess(holdout_df, k=2, min_tcrs_amount=1)
print(X_holdout.shape)

X_holdout_scaled = scaler.transform(X_holdout)

y_pred_proba_holdout = xgb_model.predict(xgb.DMatrix(X_holdout_scaled))
# y_pred_holdout = [1 if prob > 0.5 else 0 for prob in y_pred_proba_holdout]

print(y_pred_proba_holdout[:10])

(60077, 3)
      epitope             tcr  label
0  EAAGIGILTV      CASSLGNEQF      0
1  EAAGIGILTV   CASSLGVATGELF      0
2   YLEPGPVTA   CASSLGSSYEQYF      0
3   SAYGEPRKL   CASRLWFWALEAF      0
4  EAAGIGILTV  CASSSGQGLNIQYF      0
Distinct kmers: 729
(60077, 1660)
[0.99040645 0.9504607  0.9927874  0.4515879  0.61158174 0.9215012
 0.87452716 0.32839414 0.17978652 0.5254405 ]


In [18]:
holdout_df['label'] = y_pred_proba_holdout
holdout_df.to_csv('../data/true_hold_out/tcr_split_test_pred.csv', header=False, index=False)